In [1]:
import cv2
import json
import numpy as np
import mediapipe as mp
import tensorflow as tf

tf.get_logger().setLevel('ERROR')

In [2]:
model = tf.keras.models.load_model('../Models/model.keras')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 base_model (Sequential)     (None, 64)                47616     
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
Total params: 48136 (188.03 KB)
Trainable params: 48136 (188.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
with open("../Encoded Data/label_mapping.json", 'r') as f:
    label_mapping = json.load(f)

In [3]:
hands = mp.solutions.hands.Hands()
                
def get_hands_landmarks(frame):
    results = hands.process(frame)
    landmarks = np.zeros((42, 3))
    if results.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
            if results.multi_handedness[i].classification[0].index == 0: 
                landmarks[:21, :] = np.array(
                    [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]) # right
            else:
                landmarks[21:, :] = np.array(
                    [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]) # left
    return landmarks

In [5]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    landmarks = get_hands_landmarks(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    frame = cv2.flip(frame, 1)
    prediction = model.predict(landmarks[np.newaxis, ...], verbose=0)
    label = list(label_mapping.keys())[np.argmax(prediction)]
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 0, 200), 2)
    cv2.imshow('Hand Gesture Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [9]:
cap.release()
cv2.destroyAllWindows()

---